In [48]:
# 라이브러리 호출
import numpy as np
import pandas as pd
from datetime import datetime

In [49]:
# 데이터 불러오기
rawData = pd.read_csv('TPSS_region_OD3.csv', encoding='cp949', sep=",")
rawData.head()

,Date,OA,OB,DA,DB,population
0,20230920,성동구,성수2가1동,은평구,대조동,3
1,20230920,성동구,성수2가1동,은평구,불광1동,2
2,20230920,성동구,성수2가1동,은평구,불광2동,1
3,20230920,성동구,성수2가1동,은평구,수색동,2
4,20230920,성동구,성수2가1동,은평구,신사1동,3


In [50]:
# OB 와 DB 가 같은 지역 제거
# 날짜는 20230920 인 경우만으로 필터링
# population 이 적어도 100 이 넘는 지역만
# population 의 내림차순으로 정렬하기.

filterdDf =rawData[(rawData['OB']!=rawData['DB'])&(rawData['Date']==20230920)&(rawData['population']>100)].sort_values('population', ascending= False)

In [51]:
filterdDf =filterdDf.loc[:,['OB','DB','population']]

In [52]:
# 지역과 ID를 매핑하는 딕셔너리 생성
region_id_mapping = {}
unique_regions = pd.concat([filterdDf['OB'], filterdDf['DB']]).unique()
for idx, region in enumerate(unique_regions):
    region_id_mapping[region] = idx


In [53]:
# A에서 B로 이동하는 데이터 복사
output1 = filterdDf.copy()

# B에서 A로 이동하는 데이터 복사
output2 = filterdDf.copy()
# 열 이름 바꾸기
output2.columns = ['DB', 'OB', 'population']

# 두 데이터프레임 병합
merged_df = pd.concat([output1, output2])

# Location_1 및 Location_2 열의 값을 ID로 바꾸기
merged_df['DB'] = merged_df['DB'].map(region_id_mapping)
merged_df['OB'] = merged_df['OB'].map(region_id_mapping)

# 동일한 출발지와 도착지를 가진 데이터 합치기
result_df = merged_df.groupby(['DB', 'OB']).sum().reset_index()

# (Location_1=지역1 & Location_2=지역3)와 (Location_1=지역3 & Location_2=지역1)을 같은 것으로 처리
result_df['Unique_DB'] = result_df[['DB', 'OB']].min(axis=1)
result_df['Unique_OB'] = result_df[['DB', 'OB']].max(axis=1)

# 중복된 데이터 제거
result_df = result_df.drop_duplicates(subset=['Unique_DB', 'Unique_OB'])

# 불필요한 열 제거
result_df = result_df.drop(columns=['Unique_DB', 'Unique_OB'])



In [54]:
# 고유 ID와 지역 매핑 정보를 저장할 데이터프레임 생성
region_id_df = pd.DataFrame(list(region_id_mapping.items()), columns=['Region', 'ID'])

# 데이터 타입 추가
region_id_df['DataType'] = "FILE"

# 랜덤 이름을 추가
users = ['Admin','JH','JY','MS','SY','YS','Gasfard', 'Pierre', 'Santiago','Pel','Royd','Dhino','Charlie']
region_id_df['User'] = np.random.choice(users, len(region_id_df))

# 현재 시간을 얻어 'CreatedTime' 열에 추가
region_id_df['CreatedTime'] = datetime.now()

# Region 값을 가져와 설명을 추가
region_id_df['description'] = region_id_df['Region'] + '에 대한 설명'

# 해시태그 키워드
region_id_df['Keyword'] = [f"[{region}, '지역', '수도권']" for region in region_id_df['Region']]


region_id_df = region_id_df[['ID', 'Region','DataType', 'User', 'CreatedTime', 'description', 'Keyword']]



In [55]:
result_df['similarity'] = result_df['population']/result_df.max()['population']
result_df=result_df.drop(columns=['population'])

In [56]:
result_df.sort_values('similarity', ascending= False)

,DB,OB,similarity
1632,7,244,1.000000
52,1,4,0.950253
318,2,6,0.928883
561,3,7,0.919692
51,1,3,0.878906
...,...,...,...
13400,142,418,0.011604
5954,36,101,0.011604
506,2,303,0.011604
6279,39,274,0.011604


In [57]:
region_id_df

,ID,Region,DataType,User,CreatedTime,description,Keyword
0,0,대학동,FILE,Pel,2023-10-26 01:13:27.104357,대학동에 대한 설명,"[대학동, '지역', '수도권']"
1,1,명동,FILE,YS,2023-10-26 01:13:27.104357,명동에 대한 설명,"[명동, '지역', '수도권']"
2,2,서초2동,FILE,Charlie,2023-10-26 01:13:27.104357,서초2동에 대한 설명,"[서초2동, '지역', '수도권']"
3,3,회현동,FILE,JH,2023-10-26 01:13:27.104357,회현동에 대한 설명,"[회현동, '지역', '수도권']"
4,4,서교동,FILE,Pel,2023-10-26 01:13:27.104357,서교동에 대한 설명,"[서교동, '지역', '수도권']"
...,...,...,...,...,...,...,...
613,613,상대원2동,FILE,SY,2023-10-26 01:13:27.104357,상대원2동에 대한 설명,"[상대원2동, '지역', '수도권']"
614,614,갈산동,FILE,Dhino,2023-10-26 01:13:27.104357,갈산동에 대한 설명,"[갈산동, '지역', '수도권']"
615,615,수택2동,FILE,Pierre,2023-10-26 01:13:27.104357,수택2동에 대한 설명,"[수택2동, '지역', '수도권']"
616,616,귀인동,FILE,Pierre,2023-10-26 01:13:27.104357,귀인동에 대한 설명,"[귀인동, '지역', '수도권']"


In [58]:
# CSV 파일로 저장
result_df.to_csv('RelationAnalysis.csv', index=False)


In [59]:
# CSV 파일로 지역과 ID 매핑 정보 저장
region_id_df.to_csv('ID_DO_Mapping.csv', index=False)

In [62]:
# 예시 데이터프레임 생성
data = {
    'Region': ['대학동', '서교동', '상대원2동', '귀인동'],
    'Value': [100, 200, 150, 50]
}

df = pd.DataFrame(data)

# 'region_id_df'에서 Region과 ID 값만 선택
region_id_mapping = region_id_df[['ID', 'Region']]

# 'df'와 'region_id_mapping'을 병합하여 Region 값을 ID로 변환
# result_df = df.merge(region_id_mapping, left_on='Region', right_on='Region', how='left')

pd.merge(result_df, region_id_mapping, left_on = 'Region', right_on = 'Region', how='left')

# 결과를 출력
print(result_df)

  Region  Value   ID
0    대학동    100    0
1    서교동    200    4
2  상대원2동    150  613
3    귀인동     50  616
